In [32]:
!pip install pycryptodome

In [31]:
model_path = "/kaggle/input/embedder/embedding_only_model.h5"

In [29]:
import os
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
from tensorflow.keras.models import load_model
from sklearn.metrics.pairwise import euclidean_distances as L2
from tensorflow.keras.regularizers import l2
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import MobileNetV2
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm  # Import tqdm for the progress bar

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

2024-06-18 19:41:12.274409: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-18 19:41:12.274486: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-18 19:41:12.276372: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [33]:
embedder = load_model(model_path)

In [45]:
import sqlite3
import numpy as np
from PIL import Image
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
import math 
import cv2
import re
from tqdm import tqdm

sns.set_style('dark')

# Create a connection to the SQLite database
conn = sqlite3.connect('facial_recognition.db')
cursor = conn.cursor()

# Create a table for storing user embeddings
cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT NOT NULL,
    embedding BLOB NOT NULL
)
''')
conn.commit()

In [35]:
# Function to encrypt data
def encrypt_data(data, key):
    iv = get_random_bytes(AES.block_size)  # Generate a new IV for each encryption
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ct_bytes = cipher.encrypt(pad(data, AES.block_size))
    return iv + ct_bytes  # Return IV + cipher text

# Function to decrypt data
def decrypt_data(encrypted_data, key):
    iv = encrypted_data[:AES.block_size]  # Extract the IV from the start of the data
    ct = encrypted_data[AES.block_size:]  # Extract the cipher text
    cipher = AES.new(key, AES.MODE_CBC, iv)
    try:
        pt = unpad(cipher.decrypt(ct), AES.block_size)
        return pt
    except ValueError as e:
        print("Decryption error:", e)
        return None

In [36]:
def load_and_preprocess_image(image_path):
    with Image.open(image_path) as img:
        img = img.resize((224, 224))
        img = np.array(img.convert('RGB'), dtype=np.float32)
        img = (img - 127.5) / 128.0  # Normalize to [-1, 1]
    return img

import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Function to process an image and get its embedding
def get_embedding(image_path, embedding_model):
    # Load the image file, resize it, and convert to array
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to [0, 1]

    # Generate embedding
    embedding = embedding_model.predict(img_array)
    return embedding



def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [52]:
def euclidean(x1,y1,x2,y2):
    return ((x1-x2)**2 + (y1-y2)**2)**0.5

def align(x1,y1,x2,y2,img):
    x3,y3 = 0,0
    adj,hyp = 0,euclidean(x1,y1,x2,y2)
    angle = 0
    
    if y1 > y2:
        x3,y3 = x1,y2
        adj = euclidean(x3,y3,x2,y2)
        angle = -math.degrees(math.acos(adj/hyp))
    else:
        x3,y3 = x2,y1
        adj = euclidean(x3,y3,x1,y1)
        angle = math.degrees(math.acos(adj/hyp))
        
    M = cv2.getRotationMatrix2D((img.shape[1]//2,img.shape[0]//2), angle, 1)
    out = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
    return out

def read_image(directory,image_path):
    img = load_img(os.path.join(directory,image_path))
    img = img_to_array(img)
    faces = detector.detect_faces(img)
    x1,y1 = faces[0]['keypoints']['left_eye']
    x2,y2 = faces[0]['keypoints']['right_eye']
    img = align(x1,y1,x2,y2,img)
    faces = detector.detect_faces(img)
    x,y,w,h = faces[0]['box']
    img = img[y:y+h,x:x+w,:]
    img /= 255.
    img = cv2.resize(img,(224,224))
    return img

In [53]:
from mtcnn import MTCNN
import math

In [54]:
detector = MTCNN()

In [69]:
directory = '/kaggle/input/testing'
Abigail_img1 = read_image(directory,'Abigail 1.jpg')
Abigail_img2 = read_image(directory,'Abigail 2.jpg')
Kizito_img1 = read_image(directory,'Kizito 1.jpg')
Kizito_img2 = read_image(directory, 'Kizito 2.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━

In [70]:
emb_kizito1 = embedder.predict(np.expand_dims(Kizito_img1,axis=0))
emb_kizito2 = embedder.predict(np.expand_dims(Kizito_img2,axis=0))
emb_abigail1 = embedder.predict(np.expand_dims(Abigail_img1,axis=0))
emb_abigail2 = embedder.predict(np.expand_dims(Abigail_img2,axis=0))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


In [71]:
Abigail_img1 = img_to_array(Abigail_img1)
Abigail_img2 = img_to_array(Abigail_img2)
Kizito_img1 = img_to_array(Kizito_img1)
Kizito_img2 = img_to_array(Kizito_img2)

In [74]:
emb_kizito1 = embedder.predict(np.array(Kizito_img1))
emb_kizito2 = embedder.predict(Kizito_img2)
emb_abigail1 = embedder.predict(Abigail_img1)
emb_abigail2 = embedder.predict(Abigail_img2)

ValueError: as_list() is not defined on an unknown TensorShape.

In [58]:
from sklearn.metrics.pairwise import euclidean_distances as L2, cosine_similarity as cs

In [75]:
import os
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.models import load_model
from sklearn.metrics.pairwise import euclidean_distances as L2
import math

# Initialize the face detector
detector = MTCNN()

# Function to compute Euclidean distance
def euclidean(x1, y1, x2, y2):
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

# Function to align the face
def align(x1, y1, x2, y2, img):
    x3, y3 = 0, 0
    adj, hyp = 0, euclidean(x1, y1, x2, y2)
    angle = 0
    
    if y1 > y2:
        x3, y3 = x1, y2
        adj = euclidean(x3, y3, x2, y2)
        angle = -math.degrees(math.acos(adj / hyp))
    else:
        x3, y3 = x2, y1
        adj = euclidean(x3, y3, x1, y1)
        angle = math.degrees(math.acos(adj / hyp))
        
    M = cv2.getRotationMatrix2D((img.shape[1] // 2, img.shape[0] // 2), angle, 1)
    out = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
    return out

# Function to read and preprocess image
def read_image(directory, image_path):
    img = load_img(os.path.join(directory, image_path))
    img = img_to_array(img)
    faces = detector.detect_faces(img)
    x1, y1 = faces[0]['keypoints']['left_eye']
    x2, y2 = faces[0]['keypoints']['right_eye']
    img = align(x1, y1, x2, y2, img)
    faces = detector.detect_faces(img)
    x, y, w, h = faces[0]['box']
    img = img[y:y + h, x:x + w, :]
    img /= 255.0
    img = cv2.resize(img, (224, 224))
    return img

# Load the embedding model
model_path = "/kaggle/input/embedder/embedding_only_model.h5"
embedder = load_model(model_path, compile=False)

# Directory containing the images
directory = '/kaggle/input/testing'

# Read and preprocess images
Abigail_img1 = read_image(directory, 'Abigail 1.jpg')
Abigail_img2 = read_image(directory, 'Abigail 2.jpg')
Kizito_img1 = read_image(directory, 'Kizito 1.jpg')
Kizito_img2 = read_image(directory, 'Kizito 2.jpg')

# Compute embeddings
emb_kizito1 = embedder.predict(np.expand_dims(Kizito_img1, axis=0))
emb_kizito2 = embedder.predict(np.expand_dims(Kizito_img2, axis=0))
emb_abigail1 = embedder.predict(np.expand_dims(Abigail_img1, axis=0))
emb_abigail2 = embedder.predict(np.expand_dims(Abigail_img2, axis=0))

# Compute Euclidean distance
distance_kizito = L2(emb_kizito1.reshape(1, -1), emb_kizito2.reshape(1, -1))
distance_abigail = L2(emb_abigail1.reshape(1, -1), emb_abigail2.reshape(1, -1))

print("Distance between Kizito images:", distance_kizito)
print("Distance between Abigail images:", distance_abigail)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━

In [78]:
# distance_kizito = L2(emb_kizito1.reshape(1, -1), emb_abigail1.reshape(1, -1))
distance_abigail = L2(emb_abigail1.reshape(1, -1), emb_abigail2.reshape(1, -1))



# print("Distance between Kizito images:", distance_kizito)
print("Distance between Abigail images:", distance_abigail)

Distance between Abigail images: [[0.14541507]]


In [59]:
L2(Kizito_img1, Kizito_img2)

ValueError: Found array with dim 3. check_pairwise_arrays expected <= 2.

In [17]:
def register_user(username, image_path, key,embedder):
    embedding = get_embedding(image_path, embedder)
    embedding_bytes = embedding.tobytes()
    encrypted_embedding = encrypt_data(embedding_bytes, key)

    # Store username and encrypted embedding in the database
    cursor.execute('INSERT INTO users (username, embedding) VALUES (?, ?)', (username, encrypted_embedding))
    conn.commit()
    print(f"User {username} registered successfully.")


In [23]:
# Example usage
key = get_random_bytes(16)  # AES key
register_user('Caleb', '/kaggle/input/celeba-face-recognition-triplets/CelebA FR Triplets/CelebA FR Triplets/images/110393.jpg', key, embedder)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
User Caleb registered successfully.


In [24]:
def authenticate_user(username, image_path, key, embedder, threshold=1.3):
    current_embedding = get_embedding(image_path, embedder)

    # Retrieve the stored embedding from the database
    cursor.execute('SELECT embedding FROM users WHERE username = ?', (username,))
    result = cursor.fetchone()

    if result:
        stored_encrypted_embedding = result[0]
        stored_embedding_bytes = decrypt_data(stored_encrypted_embedding, key)
        if stored_embedding_bytes:
            stored_embedding = np.frombuffer(stored_embedding_bytes, dtype=current_embedding.dtype)
            dist = L2(current_embedding.reshape(1, -1), stored_embedding.reshape(1, -1))[0][0]
            print("Distance:", dist)
            if dist <= threshold:
                print(f"User {username} authenticated successfully.")
            else:
                print(f"Authentication failed for user {username}.")
        else:
            print("Decryption error occurred.")
    else:
        print(f"User {username} not found.")

# Example usage
authenticate_user('Caleb', '/kaggle/input/celeba-face-recognition-triplets/CelebA FR Triplets/CelebA FR Triplets/images/105432.jpg', key, embedder)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Distance: 0.093240954
User Caleb authenticated successfully.


In [27]:
authenticate_user('Daniel', '/kaggle/input/celeba-face-recognition-triplets/CelebA FR Triplets/CelebA FR Triplets/images/107918.jpg', key, embedder)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Decryption error: Padding is incorrect.
Decryption error occurred.


In [12]:
import os
import pandas as pd

# Path to your dataset CSV and images directory
csv_path = '/kaggle/input/celeba-face-recognition-triplets/CelebA FR Triplets/CelebA FR Triplets/triplets.csv'
image_dir = '/kaggle/input/celeba-face-recognition-triplets/CelebA FR Triplets/CelebA FR Triplets/images'

# Load the triplets CSV
triplets_df = pd.read_csv(csv_path)
triplets_df2 = triplets_df.iloc[:40]  # Select first 40 rows

# Extract 7 file paths of anchors, positives, and negatives
def get_file_paths(df, image_dir, num_samples=7):
    file_paths = []
    for index, row in df.iterrows():
        if index >= num_samples:
            break
        anchor_path = os.path.join(image_dir, row['anchor'])
        positive_path = os.path.join(image_dir, row['pos'])
        negative_path = os.path.join(image_dir, row['neg'])
        file_paths.append((anchor_path, positive_path, negative_path))
    return file_paths

file_paths = get_file_paths(triplets_df2, image_dir, num_samples=7)

# Print the file paths
for i, paths in enumerate(file_paths):
    anchor, positive, negative = paths
    print(f"Sample {i + 1}:")
    print(f"  Anchor: {anchor}")
    print(f"  Positive: {positive}")
    print(f"  Negative: {negative}")
    print()


Sample 1:
  Anchor: /kaggle/input/celeba-face-recognition-triplets/CelebA FR Triplets/CelebA FR Triplets/images/056279.jpg
  Positive: /kaggle/input/celeba-face-recognition-triplets/CelebA FR Triplets/CelebA FR Triplets/images/108998.jpg
  Negative: /kaggle/input/celeba-face-recognition-triplets/CelebA FR Triplets/CelebA FR Triplets/images/030848.jpg

Sample 2:
  Anchor: /kaggle/input/celeba-face-recognition-triplets/CelebA FR Triplets/CelebA FR Triplets/images/024091.jpg
  Positive: /kaggle/input/celeba-face-recognition-triplets/CelebA FR Triplets/CelebA FR Triplets/images/000023.jpg
  Negative: /kaggle/input/celeba-face-recognition-triplets/CelebA FR Triplets/CelebA FR Triplets/images/093653.jpg

Sample 3:
  Anchor: /kaggle/input/celeba-face-recognition-triplets/CelebA FR Triplets/CelebA FR Triplets/images/122082.jpg
  Positive: /kaggle/input/celeba-face-recognition-triplets/CelebA FR Triplets/CelebA FR Triplets/images/045833.jpg
  Negative: /kaggle/input/celeba-face-recognition-trip

In [4]:
!pip install streamlit mtcnn opencv-python-headless pycryptodome


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 54.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 47.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 87.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 4.6 MB/s eta 0:00:00


In [5]:
!pip install pyngrok
from pyngrok import ngrok


In [34]:
%%writefile app.py
import streamlit as st
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.models import load_model
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import sqlite3

# Function to encrypt data
def encrypt_data(data, key):
    iv = get_random_bytes(AES.block_size)  # Generate a new IV for each encryption
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ct_bytes = cipher.encrypt(pad(data, AES.block_size))
    return iv + ct_bytes  # Return IV + cipher text

# Function to decrypt data
def decrypt_data(encrypted_data, key):
    iv = encrypted_data[:AES.block_size]  # Extract the IV from the start of the data
    ct = encrypted_data[AES.block_size:]  # Extract the cipher text
    cipher = AES.new(key, AES.MODE_CBC, iv)
    try:
        pt = unpad(cipher.decrypt(ct), AES.block_size)
        return pt
    except ValueError as e:
        print("Decryption error:", e)
        return None

# Function to load and preprocess image
def load_and_preprocess_image(image):
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = np.array(img, dtype=np.float32)
    img = (img - 127.5) / 128.0  # Normalize to [-1, 1]
    return np.expand_dims(img, axis=0)

# Function to get embedding
def get_embedding(image, model):
    preprocessed_image = load_and_preprocess_image(image)
    embedding = model.predict(preprocessed_image)
    return embedding

# Function to capture image using webcam
def capture_image():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            continue
        cv2.imshow('Press "s" to capture and "q" to quit', frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('s'):
            captured_image = frame
            break
        elif key == ord('q'):
            captured_image = None
            break
    cap.release()
    cv2.destroyAllWindows()
    return captured_image

# Function to detect face using MTCNN
def detect_face(image):
    detector = MTCNN()
    result = detector.detect_faces(image)
    if result:
        bounding_box = result[0]['box']
        x, y, w, h = bounding_box
        face = image[y:y+h, x:x+w]
        return face
    return None

# Database setup
conn = sqlite3.connect('facial_recognition.db')
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT NOT NULL,
    embedding BLOB NOT NULL
)
''')
conn.commit()

# Load the embedding model
model_path = "/kaggle/input/embedder/embedding_only_model.h5"
embedding_model = load_model(model_path, compile=False)

# Streamlit app
st.title("Facial Recognition System")

key = get_random_bytes(16)  # AES key

# Login/Register
choice = st.sidebar.selectbox("Login/Register", ["Login", "Register"])

if choice == "Register":
    st.header("Register")
    username = st.text_input("Enter Username")
    if st.button("Capture and Register"):
        image = capture_image()
        if image is not None:
            face = detect_face(image)
            if face is not None:
                embedding = get_embedding(face, embedding_model)
                embedding_bytes = embedding.tobytes()
                encrypted_embedding = encrypt_data(embedding_bytes, key)
                cursor.execute('INSERT INTO users (username, embedding) VALUES (?, ?)', (username, encrypted_embedding))
                conn.commit()
                st.success(f"User {username} registered successfully.")
            else:
                st.error("No face detected. Please try again.")
        else:
            st.error("Image capture failed. Please try again.")

elif choice == "Login":
    st.header("Login")
    username = st.text_input("Enter Username")
    if st.button("Capture and Authenticate"):
        image = capture_image()
        if image is not None:
            face = detect_face(image)
            if face is not None:
                current_embedding = get_embedding(face, embedding_model)
                cursor.execute('SELECT embedding FROM users WHERE username = ?', (username,))
                result = cursor.fetchone()
                if result:
                    stored_encrypted_embedding = result[0]
                    stored_embedding_bytes = decrypt_data(stored_encrypted_embedding, key)
                    if stored_embedding_bytes:
                        stored_embedding = np.frombuffer(stored_embedding_bytes, dtype=current_embedding.dtype)
                        dist = L2(current_embedding.reshape(1, -1), stored_embedding.reshape(1, -1))[0][0]
                        if dist <= 1.3:
                            st.success(f"User {username} authenticated successfully.")
                        else:
                            st.error(f"Authentication failed for user {username}.")
                    else:
                        st.error("Decryption error occurred.")
                else:
                    st.error(f"User {username} not found.")
            else:
                st.error("No face detected. Please try again.")
        else:
            st.error("Image capture failed. Please try again.")


Overwriting app.py


In [27]:
%%writefile app.py
import streamlit as st
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.models import load_model
import sqlite3
from sklearn.metrics.pairwise import euclidean_distances as L2
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Function to load and preprocess image
def load_and_preprocess_image(image):
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = np.array(img, dtype=np.float32)
    img = (img - 127.5) / 128.0  # Normalize to [-1, 1]
    return np.expand_dims(img, axis=0)

# def load_and_preprocess_image(image):
#         img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         img = cv2.resize(img, (224, 224))
#         img = np.array(img, dtype=np.float32)
#         img = (img - 127.5) / 128.0  # Normalize to [-1, 1]
#         return img


def get_embedding(image, embedding_model):
    preprocessed_image = load_and_preprocess_image(image)
    # Generate embedding
    embedding = embedding_model.predict(preprocessed_image)
    return embedding

# # Function to get embedding
# def get_embedding(image, model):
#     preprocessed_image = load_and_preprocess_image(image)
#     embedding = model.predict(preprocessed_image)
#     return embedding

# Function to detect face using MTCNN
def detect_face(image):
    detector = MTCNN()
    result = detector.detect_faces(image)
    if result:
        bounding_box = result[0]['box']
        x, y, w, h = bounding_box
        face = image[y:y+h, x:x+w]
        return face
    return None

# Database setup
conn = sqlite3.connect('facial_recognition.db')
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT NOT NULL,
    embedding BLOB NOT NULL
)
''')
conn.commit()

# Load the embedding model
model_path = "/kaggle/input/embedder/embedding_only_model.h5"
embedding_model = load_model(model_path, compile=False)

# Streamlit app
st.title("Facial Recognition System")

# Login/Register
choice = st.sidebar.selectbox("Register/Login", [ "Register", "Login"])

if choice == "Register":
    st.header("Register")
    username = st.text_input("Enter Username")
    uploaded_file = st.file_uploader("Upload a face image", type=["jpg", "jpeg", "png"])
    if st.button("Register") and uploaded_file is not None:
        image = cv2.imdecode(np.frombuffer(uploaded_file.read(), np.uint8), 1)
        face = detect_face(image)
        if face is not None:
            embedding = get_embedding(face, embedding_model)
            embedding_bytes = embedding.tobytes()
            cursor.execute('INSERT INTO users (username, embedding) VALUES (?, ?)', (username, embedding_bytes))
            conn.commit()
            st.success(f"User {username} registered successfully.")
        else:
            st.error("No face detected. Please try again.")
elif choice == "Login":
    st.header("Login")
    username = st.text_input("Enter Username")
    uploaded_file = st.file_uploader("Upload a face image", type=["jpg", "jpeg", "png"])
    if st.button("Login") and uploaded_file is not None:
        image = cv2.imdecode(np.frombuffer(uploaded_file.read(), np.uint8), 1)
        face = detect_face(image)
        if face is not None:
            current_embedding = get_embedding(face, embedding_model)
            cursor.execute('SELECT embedding FROM users WHERE username = ?', (username,))
            result = cursor.fetchone()
            if result:
                stored_embedding_bytes = result[0]
                stored_embedding = np.frombuffer(stored_embedding_bytes, dtype=current_embedding.dtype)
                
                # Check for finite values in embeddings
                if not np.all(np.isfinite(current_embedding)):
                    st.error("Non-finite values in current embedding.")
                elif not np.all(np.isfinite(stored_embedding)):
                    st.error("Non-finite values in stored embedding.")
                else:
                    dist = L2(current_embedding.reshape(1, -1), stored_embedding.reshape(1, -1))[0][0]
                    if dist <= 1.3:
                        st.success(f"User {username} authenticated successfully.")
                    else:
                        st.error(f"Authentication failed for user {username}.")
            else:
                st.error(f"User {username} not found.")
        else:
            st.error("No face detected. Please try again.")


Overwriting app.py


In [21]:
!ngrok config add-authtoken 2VJXf1SjlcpG5WPLyrgDcEaM2ct_7KytqnABitp5VfqfxZCYf

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [22]:

!wget -q -O - ipv4.icanhazip.com

35.236.179.101


In [23]:
!npm install -g localtunnel


m##################] - reify:yargs: timing reifyNode:node_modules/localtunnel/nmpleted in
changed 22 packages in 1s

3 packages are looking for funding
  run `npm fund` for details


In [28]:
!streamlit run app.py & npx localtunnel --port 8501





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.19.2.2:8501
  External URL: http://35.236.179.101:8501

your url is: https://slick-colts-slide.loca.lt
2024-06-18 19:30:12.123817: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-18 19:30:12.123893: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-18 19:30:12.125756: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0